<a href="https://colab.research.google.com/github/abhranil-datascience/ClassificationPOC/blob/master/XGBClassifierPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/ML/Classification/XGBClassifier')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
########### Import Dataset #############
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
X=dataset.iloc[:,1:4].values
Y=dataset.iloc[:,4].values
########### Categorical Variable ###############
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,0]=le.fit_transform(X[:,0])
########## Feature Scaling ##############
from sklearn.preprocessing import MinMaxScaler
SC=MinMaxScaler(feature_range=(0,1))
XScaled=SC.fit_transform(X)
########## Train Test Split #############
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(XScaled,Y,test_size=0.1,random_state=42)
########## Check VIF ####################
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=[0]*XTrain.shape[1]
for col in range(0,XTrain.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrain,exog_idx=col)#1.6142518559270171, 2.9082305467018816, 2.753146445004674
VIF

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


[1.6142518559270171, 2.9082305467018816, 2.753146445004674]

In [0]:
######### Grid Search CV ############
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
XGBClassifier_GS=XGBClassifier()
params_XGB=[{'max_depth':[3,4,5],
            'learning_rate':[0.00001,0.00005,0.0001],
            'n_estimators':[1,5,10,15,20,25,30,35,40,45,50],
            'booster':['gbtree','gblinear','dart'],
            'gamma':[0.00001,0.00005,0.0001,0.0005]}]
RegressorXGB=XGBClassifier()
GS=GridSearchCV(estimator=RegressorXGB,param_grid=params_XGB,scoring='accuracy',cv=3)
GS.fit(XTrain,YTrain)
best_params=GS.best_params_
best_params

{'booster': 'gbtree',
 'gamma': 1e-05,
 'learning_rate': 1e-05,
 'max_depth': 3,
 'n_estimators': 1}

In [0]:
######### KFold ############
from sklearn.model_selection import KFold
kf=KFold(n_splits=40,shuffle=True)
from sklearn import metrics as mt
import statistics as st
XGB=XGBClassifier(booster='gbtree',gamma=1e-05,learning_rate=1e-05,max_depth=3,n_estimators=1)
Accuracies=[]
for count in range(0,100):
  for train_index,val_index in kf.split(XTrain):
    XTrainKF,YTrainKF=XTrain[train_index],YTrain[train_index]
    XValKF,YValKF=XTrain[val_index],YTrain[val_index]
    XGB.fit(XTrainKF,YTrainKF)
    YPredKF=XGB.predict(XValKF)
    Accuracies.append(mt.accuracy_score(YValKF,YPredKF))
print("Average Accuracy: "+str(st.mean(Accuracies)))#0.9074722222222222

Average Accuracy: 0.9075833333333333


In [0]:
########### Predict ################
XGB.fit(XTrain,YTrain)
YPred=XGB.predict(XTest)
print("Accuracy: "+str(mt.accuracy_score(YTest,YPred)))#0.925
cm=mt.confusion_matrix(YTest,YPred)
cm

Accuracy: 0.925


array([[24,  3],
       [ 0, 13]])